## Hyperparameters selection with bayesian optimization (GPU)

### Initialization

In [ ]:
# Read and save on google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
root_path = 'drive/My Drive/M2/' 
os.chdir(root_path)

In [ ]:
# Libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import f1_score
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from xgboost import plot_importance
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
import graphviz
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.utils.data import Dataset, DataLoader
import torchvision.datasets as datasets

import sklearn
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Read data
df = pd.read_csv("data.csv")
X = df.iloc[:,1:-1]
y = np.array(df["y"])
y = (y == 1)*1
n,d = X.shape

In [ ]:
# Train/test split 
X_train, X_test, y_train, y_test = train_test_split(X.values, y, test_size=0.2)

scale = StandardScaler()
X_train = scale.fit_transform(X_train)
X_test = scale.transform(X_test)

X_train = torch.from_numpy(X_train).to(torch.float)
X_test = torch.from_numpy(X_test).to(torch.float)
y_train = torch.from_numpy(y_train).to(torch.float)
y_test = torch.from_numpy(y_test).to(torch.float)
ntest = y_test.shape[0]

### Dataset, dataloader and model

In [ ]:
class EEG_dataset(Dataset):
    
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __getitem__(self, index):
        return self.X[index,:], self.y[index]
    
    def __len__(self):
        return self.X.shape[0]

In [ ]:
dataset = EEG_dataset(X_train, y_train)
dataset_test = EEG_dataset(X_test, y_test)
batch_size = 256
dataloader = DataLoader(dataset, shuffle=True, batch_size = batch_size)
dataloader_test = DataLoader(dataset_test, shuffle=False, batch_size = ntest)

In [ ]:
# Convolution neural net for classification task

class Flatten(nn.Module):
    def forward(self, x):
        batch_size = x.shape[0]
        return x.view(batch_size, -1)
    

class ConvNet(nn.Module):
    def __init__(self, out_channels, inSize = 178, kernel_size=3, stride=1):
        super(ConvNet, self).__init__()
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        
        # convolution + max pooling
        self.main = nn.Sequential(
            nn.Conv1d(1, out_channels, kernel_size, stride),
            nn.MaxPool1d(kernel_size, stride),
            nn.Conv1d(out_channels, out_channels*2, kernel_size, stride+1),
            nn.MaxPool1d(kernel_size, stride+1),
            nn.Conv1d(out_channels*2, out_channels*4, kernel_size, stride+1),
            nn.MaxPool1d(kernel_size, stride+1),
            nn.AdaptiveMaxPool1d(1), # output of size N x out_channels*4 x 1
            Flatten() # for the final linear layer
            
        )

        
        size=out_channels*4
        self.lin = nn.Linear(size,1)
        
        
        
    def forward(self, x):
        conv_flatt = self.main(x)
        lin = torch.sigmoid(self.lin(conv_flatt))
        return lin.squeeze()
    
    
    def predict(self, xtest):
        xtest = xtest.reshape(xtest.shape[0],1,178)
        ypred = self.forward(xtest)
        ypred = (ypred >= 0.5).to(torch.float)
        return ypred

### Bayesian optimization

In [ ]:
from hyperopt import fmin, tpe, hp

nepochs=200


# Input : Hyperparameters to optimize (learning rate and size of convolution)
# Output : (- Accuracy)

def loss(space):
    lr = space['lr']
    hlist = space['hlist']
    
    model = ConvNet(hlist).to(device)
    optim = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.BCELoss(reduction = "mean")

    # 1) Train
    for epoch in range(nepochs):
        model.train()
        for x,y in dataloader:
            x = x.reshape(x.shape[0],1,178).to(device) 
            y = y.to(device)
            ypred = model(x)
            l = loss_fn(ypred,y)
            optim.zero_grad()
            l.backward()
            optim.step()
        
    # 2) Test
    model.eval()
    acc = 0.
    for xtest, ytest in dataloader_test:
        xtest = xtest.reshape(xtest.shape[0],1,178).to(device)
        ytest = ytest.to(device)
        ypred = model(xtest)
        ypred = (ypred >= 0.5).to(torch.float)
        acc += (ypred == ytest).sum().item()
    acc /= len(dataset_test)
    
    return - acc


# State space of hyperparameters
space = {
    'lr': hp.choice("lr", [0.00001, 0.0001, 0.0005, 0.001, 0.002, 0.005, 0.007, 0.0001, 0.0002, 0.0005]),
    'hlist': hp.choice("hlist", [10, 15, 20, 25, 30, 35])
}



# Return indices of best parameters found
n_eval = 10
best = fmin(
    fn=loss,
    space=space,
    algo=tpe.suggest,
    max_evals=n_eval
)

print("Best parameters after {} trials:".format(n_eval))
print(best)